In [1]:
!pip install -q streamlit datasets faiss-cpu sentence-transformers transformers accelerate pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 145.6 MB/s eta 0:00:00


In [3]:
import os, getpass
os.environ["HUGGINGFACE_TOKEN"] = getpass.getpass("HF token: ")

HF token: ··········


In [4]:
%%writefile config.py
# -*- coding: utf-8 -*-
import os

# Hugging Face token
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

# Model ayarları
EMBED_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
GEN_MODEL = "microsoft/Phi-3-mini-4k-instruct"

# Dataset URL (Hugging Face)
DATA_URL = "https://huggingface.co/datasets/mertbozkurt/turkish-recipe/resolve/main/datav2.csv"


Writing config.py


In [2]:
import torch
print("GPU aktif mi?", torch.cuda.is_available())
print("GPU adı:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Yok")

GPU aktif mi? True
GPU adı: Tesla T4


In [5]:
%%writefile rag_utils.py
# -*- coding: utf-8 -*-
import requests
import pandas as pd
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import re
from config import HF_TOKEN, EMBED_MODEL, GEN_MODEL, DATA_URL

# -------------------------------------------------------------
# MODEL YÜKLEME
# -------------------------------------------------------------
def load_models():
    print("🔄 Modeller yükleniyor...")
    embedder = SentenceTransformer(EMBED_MODEL)

    tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL)
    model = AutoModelForCausalLM.from_pretrained(
        GEN_MODEL,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )

    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=400,
        temperature=0.3,
        top_p=0.9,
        do_sample=False,
    )
    print("✅ Modeller hazır.")
    return embedder, generator


# -------------------------------------------------------------
# VERİ SETİ YÜKLEME
# -------------------------------------------------------------
def load_recipes():
    print("🍽️ Veri seti indiriliyor...")
    headers = {"Authorization": f"Bearer {HF_TOKEN}"} if HF_TOKEN else {}
    response = requests.get(DATA_URL, headers=headers)
    response.raise_for_status()

    with open("turkish_recipe.csv", "wb") as f:
        f.write(response.content)

    df = pd.read_csv("turkish_recipe.csv")
    print(f"✅ {len(df)} tarif yüklendi.")
    return df


# -------------------------------------------------------------
# INDEX OLUŞTURMA (FAISS + TF-IDF hibrit)
# -------------------------------------------------------------
# -------------------------------------------------------------
# INDEX OLUŞTURMA
# -------------------------------------------------------------
def create_index(df, embedder):
    texts = []
    for _, row in df.iterrows():
        title = (
            row.get("Title")
            or row.get("title")
            or row.get("Name")
            or row.get("name")
            or "Bilinmeyen Tarif"
        )
        category = row.get("Category") or row.get("category") or ""
        materials = (
            row.get("Materials")
            or row.get("materials")
            or row.get("Ingredients")
            or row.get("ingredients")
            or ""
        )
        how_to_do = (
            row.get("How-to-do")
            or row.get("how_to_do")
            or row.get("Instructions")
            or row.get("instructions")
            or row.get("How-to-Do")
            or ""
        )
        question = row.get("Question") or row.get("question") or ""

        combined = f"Tarif Adı: {title}\nKategori: {category}\nMalzemeler: {materials}\nYapılışı: {how_to_do}\nSoru: {question}".strip()
        if len(combined) > 30:
            texts.append(combined)

    if not texts:
        raise ValueError("❌ Veri kümesinde geçerli tarif bulunamadı. CSV sütun adlarını kontrol et!")

    print("🧠 Embedding hesaplanıyor...")
    embeddings = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=True)
    embeddings = np.array(embeddings)
    faiss.normalize_L2(embeddings)
    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)
    print(f"✅ Hibrit index hazır ({len(texts)} tarif).")

    # 👉 Sadece 2 değer döndür: app.py ile uyumlu hale getiriyoruz
    return index, texts



# -------------------------------------------------------------
# RAG CEVAP (Hibrit + başlık kontrolü)
# -------------------------------------------------------------
# -------------------------------------------------------------
# RAG CEVAP (opsiyonel hibrit parametrelerle uyumlu)
# -------------------------------------------------------------
def rag_answer(question, embedder, generator, index, texts, metas=None, tfidf=None, X=None):
    """
    question, embedder, generator, index, texts ZORUNLU.
    metas, tfidf, X opsiyoneldir (hibrit arama kurmak istersen kullanılır).
    Parametreler gelmezse yalnızca FAISS (dense) aramayı kullanır.
    """
    try:
        # 1) FAISS ile semantik arama
        qvec = embedder.encode([question], convert_to_numpy=True)
        faiss.normalize_L2(qvec)
        D, I = index.search(qvec, 10)  # biraz yüksek tutup filtreleyeceğiz

        # 2) Basit anahtar-kelime sinyali (soru kelimeleriyle kaba eşleştirme)
        q_tokens = {t for t in question.lower().split() if len(t) > 2}

        candidates = []
        for rank, (idx, score) in enumerate(zip(I[0], D[0])):
            if idx == -1:
                continue
            text = texts[idx]
            t_low = text.lower()
            kw_hit = any(k in t_low for k in q_tokens)
            # Skor + anahtar kelime eşleşmesiyle basit filtre
            if score > 0.35 or kw_hit or rank < 3:
                candidates.append((idx, float(score), text))

        if not candidates:
            # hiçbiri geçmezse en yakını al
            candidates = [(I[0][0], float(D[0][0]), texts[I[0][0]])]

        # 3) (OPSİYONEL) TF-IDF ile hibrit skor (tfidf ve X verilmişse)
        # Not: app.py şu an bunları göndermiyor; None ise bu blok atlanır.
        if tfidf is not None and X is not None:
            try:
                # Soru için tf-idf vektörü ve cosine benzerlik
                q_tfidf = tfidf.transform([question])
                from sklearn.metrics.pairwise import cosine_similarity
                # Aday indeksleri
                cand_indices = [ci for ci, _, _ in candidates]
                sim = cosine_similarity(q_tfidf, X[cand_indices]).ravel()
                # Basit hibrit: 0.6 * dense + 0.4 * tfidf
                new_candidates = []
                for (ci, dscore, text), tfscore in zip(candidates, sim):
                    hybrid = 0.6 * dscore + 0.4 * float(tfscore)
                    new_candidates.append((ci, hybrid, text))
                # Hibrit skora göre sırala
                new_candidates.sort(key=lambda x: x[1], reverse=True)
                candidates = new_candidates
            except Exception:
                # Hibritte sorun olursa dense ile devam
                pass

        # 4) En iyi 2-3 bağlam
        top_contexts = [c[2] for c in candidates[:3]]
        context = "\n\n---\n\n".join(top_contexts)

        # 5) Model dostu, sade ve tutarlı çıktı formatı
        prompt = f"""
<|system|>
Sen bir Türk mutfağı uzmanısın. Aşağıdaki bilgilere dayanarak
SADECE ilgili yemeğin tarifini üret. Başlıklar şu şekilde olsun:

# {{Yemek Adı}}
## Malzemeler
- ...
## Yapılış
1) ...

Ekstra yorum yapma, başka yemeklerden bahsetme.
</s>
<|user|>
Soru: {question}

Kullanabileceğin bilgi:
{context}
</s>
<|assistant|>
"""

        out = generator(
            prompt,
            max_new_tokens=350,
            do_sample=False  # deterministik olsun
        )[0]["generated_text"]

        # Bazı modeller tag'ları olduğu gibi döndürebiliyor; yardımcının kısmını ayıkla
        if "<|assistant|>" in out:
            out = out.split("<|assistant|>", 1)[-1]

        # Çok satırlı gürültüyü kırp
        return out.strip()

    except Exception as e:
        return f"⚠️ Hata: {str(e)}"



# -------------------------------------------------------------
# TARİF ÖNERİSİ
# -------------------------------------------------------------
def suggest_recipes(ingredients_text, embedder, index, texts, top_k=5):
    try:
        query = f"Bu malzemelerle hangi yemekler yapılabilir: {ingredients_text}"
        qvec = embedder.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(qvec)
        D, I = index.search(qvec, top_k)

        suggestions = []
        for i in I[0]:
            if i != -1:
                recipe = texts[i].split("\n")[0].replace("Tarif Adı:", "").strip()
                suggestions.append(recipe)

        if not suggestions:
            return "⚠️ Uygun tarif bulunamadı."
        return "🍴 Önerilen Tarifler:\n\n" + "\n".join([f"• {r}" for r in suggestions])

    except Exception as e:
        return f"⚠️ Hata: {str(e)}"


Writing rag_utils.py


In [6]:
%%writefile app.py
# -*- coding: utf-8 -*-
import gradio as gr
from rag_utils import load_models, load_recipes, create_index, rag_answer, suggest_recipes

# -------------------------------------------------------------
# MODEL VE INDEX HAZIRLIĞI
# -------------------------------------------------------------
embedder, generator = load_models()
df = load_recipes()
index, texts = create_index(df, embedder)

# -------------------------------------------------------------
# GRADIO FONKSİYONLARI
# -------------------------------------------------------------
def chatbot_fn(question):
    try:
        return rag_answer(question, embedder, generator, index, texts)
    except Exception as e:
        import traceback
        return f"⚠️ Hata oluştu (Tarif Sorgusu):\n\n{traceback.format_exc()}"

def suggestion_fn(ingredients):
    try:
        return suggest_recipes(ingredients, embedder, index, texts)
    except Exception as e:
        import traceback
        return f"⚠️ Hata oluştu (Tarif Önerisi):\n\n{traceback.format_exc()}"

# -------------------------------------------------------------
# ÖZEL CSS
# -------------------------------------------------------------
custom_css = """
#content {
    max-width: 750px !important;
    margin: 0 auto !important;
}
#header h1, #header p {
    text-align: center !important;
}
.gradio-container {
    justify-content: center !important;
    align-items: center !important;
}
label {
    text-align: center !important;
    display: block !important;
    font-weight: 600;
    color: #4a4a4a;
}
textarea, input {
    text-align: center !important;
}
#footer {
    text-align: center !important;
    margin-top: 30px !important;
    color: gray !important;
    font-size: 13px !important;
}
"""

# -------------------------------------------------------------
# ORTALANMIŞ TASARIMLI UI
# -------------------------------------------------------------
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as demo:
    with gr.Row(elem_id="header"):
        gr.Markdown(
            """
            <div>
                <h1>🍲 Türk Lezzetleri Asistanı </h1>
                <p style='font-size:17px; color:gray;'>
                    Yemek tarifini sorabilir veya elindeki malzemelere göre öneri alabilirsin 👩‍🍳
                </p>
            </div>
            """,
        )

    # ❌ justify kaldırıldı, hizalama CSS ile yapılıyor
    with gr.Row(elem_id="main", equal_height=True):
        with gr.Column(scale=1, min_width=600, elem_id="content"):
            with gr.Tab("🔍 Tarif Sor"):
                q = gr.Textbox(
                    label="👩‍🍳 Tarif Sorusu",
                    placeholder="Örn: Karnıyarık nasıl yapılır?",
                    lines=2
                )
                out = gr.Textbox(
                    label="🍽️ Tarif Yanıtı",
                    lines=15,
                    show_copy_button=True
                )
                btn = gr.Button("Tarifi Göster", variant="primary")
                btn.click(chatbot_fn, inputs=q, outputs=out)

            with gr.Tab("🥦 Malzemeye Göre Öneri"):
                ing = gr.Textbox(
                    label="🧺 Elindeki Malzemeleri Yaz",
                    placeholder="örnek: tavuk, patates, yoğurt",
                    lines=2
                )
                out2 = gr.Textbox(
                    label="🍴 Önerilen Tarifler",
                    lines=10,
                    show_copy_button=True
                )
                btn2 = gr.Button("Tarif Öner", variant="primary")
                btn2.click(suggestion_fn, inputs=ing, outputs=out2)

    gr.Markdown(
        """
        <div id='footer'>
            © 2025 - Geliştiren: <b>İrem Sert</b> | Powered by Hugging Face + Gradio
        </div>
        """
    )

demo.launch(share=True)


Writing app.py


In [7]:
%%writefile requirements.txt
gradio
datasets
faiss-cpu
sentence-transformers
transformers
accelerate
pandas
requests


Writing requirements.txt


In [ ]:
!pip install -r requirements.txt
!python3 app.py



2025-10-21 19:19:45.711975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761074385.731363    1197 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761074385.737330    1197 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761074385.752212    1197 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761074385.752235    1197 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761074385.752239    1197 computation_placer.cc:177] computation placer alr